In [2]:
import pandas as pd

In [3]:
matches = pd.read_csv("../data/raw/matches.csv", index_col=0)

In [4]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,Match Report,NaN,13.0,1.0,17.8,0.0,0,0,2022,Liverpool
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,Match Report,NaN,25.0,9.0,16.8,1.0,0,1,2022,Liverpool
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,Match Report,NaN,9.0,4.0,17.2,1.0,0,0,2022,Liverpool
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,Match Report,NaN,17.0,4.0,14.7,0.0,0,0,2022,Liverpool
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,Match Report,NaN,16.0,5.0,15.8,0.0,0,0,2022,Liverpool


In [5]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [6]:
# Clean the data
# Drop unnnecessary columns
matches.drop(["comp", "notes"], axis=1, inplace=True)

In [7]:
# Convert data types
matches["date"] = pd.to_datetime(matches["date"])
matches['gf'] = pd.to_numeric(matches['gf'])

In [8]:
# Sort by date time descending
matches.sort_values(by="date", ascending=False, inplace=True)

In [9]:
# Calculate the rolling mean of gf of each team
# Sort the DataFrame by 'date'
matches = matches.sort_values(by='date')

# Reset the index to avoid issues with duplicate indices
matches.reset_index(drop=True, inplace=True)

# Now, calculate the rolling mean
matches['gf_avg_last_10'] = matches.groupby('team')['gf'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)
matches['sh_avg_last_10'] = matches.groupby('team')['sh'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

matches['sh_sot_ratio'] = matches['sh'] / matches['sot']
matches['sh_sot_ratio_avg_last_10'] = matches.groupby('team')['sh_sot_ratio'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

In [10]:
# Categorize data
matches["target"] = matches["result"].astype("category").cat.codes
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] =    matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [11]:
# Create a new column 'gf_avg_category_codes' by cutting 'gf_avg_last_10' into quintiles
matches['gf_avg_category_codes'] = pd.qcut(matches['gf_avg_last_10'], q=5, labels=False)
matches['sh_avg_category_codes'] = pd.qcut(matches['sh_avg_last_10'], q=5, labels=False)
matches['sh_sot_ratio_avg_category_codes'] = pd.qcut(matches['sh_sot_ratio_avg_last_10'], q=5, labels=False)

In [14]:
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City")
group

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,sh_sot_ratio,sh_sot_ratio_avg_last_10,target,venue_code,opp_code,hour,day_code,gf_avg_category_codes,sh_avg_category_codes,sh_sot_ratio_avg_category_codes
16,2022-08-07,16:30,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,2.2,...,13.000000,13.000000,2,0,21,16,6,3,2,4.0
31,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,1.7,...,2.714286,7.857143,2,1,2,15,5,4,4,4.0
55,2022-08-21,16:30,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,2.1,...,2.100000,5.938095,0,0,16,16,6,4,4,4.0
60,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,2.2,...,3.600000,5.353571,2,1,7,15,5,4,4,4.0
89,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,3.3,...,1.888889,4.660635,2,1,17,19,2,4,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,2024-02-05,20:00,Matchweek 23,Mon,Away,W,3.0,1.0,Brentford,2.1,...,1.666667,2.843232,2,0,3,20,0,4,4,0.0
1228,2024-02-10,12:30,Matchweek 24,Sat,Home,W,2.0,0.0,Everton,1.6,...,6.333333,3.156566,2,1,8,12,5,4,4,1.0
1242,2024-02-17,17:30,Matchweek 25,Sat,Home,D,1.0,1.0,Chelsea,2.6,...,6.200000,3.351566,0,1,6,17,5,4,4,2.0
1256,2024-02-20,19:30,Matchweek 18,Tue,Home,W,1.0,0.0,Brentford,2.3,...,2.500000,3.501566,2,1,3,19,1,4,4,2.0


In [22]:
def rolling_mean(df, cols, new_cols, window):
    df = df.sort_values(by='date')
    rolling_mean = df[cols].rolling(window=window, min_periods=1, closed='left').mean()
    df[new_cols] = rolling_mean
    df = df.dropna(subset=new_cols)
    return df

In [19]:
cols = ["gf","ga", "sh", "sot", "sh_sot_ratio"]
new_cols = [f"{c}_rolling_mean" for c in cols]
new_cols

['gf_rolling_mean',
 'ga_rolling_mean',
 'sh_rolling_mean',
 'sot_rolling_mean',
 'sh_sot_ratio_rolling_mean']

In [30]:
rolling_mean(group, cols, new_cols, 3)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_avg_category_codes,sh_avg_category_codes,sh_sot_ratio_avg_category_codes,gf_rolling_mean,ga_rolling_mean,sh_rolling_mean,sot_rolling_mean,sh_sot_ratio_rolling_mean
31,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,1.7,...,15,5,4,4,4.0,2.000000,0.000000,13.000000,1.000000,13.000000
55,2022-08-21,16:30,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,2.1,...,16,6,4,4,4.0,3.000000,0.000000,16.000000,4.000000,7.857143
60,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,2.2,...,15,5,4,4,4.0,3.000000,1.000000,17.666667,6.000000,5.938095
89,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,3.3,...,19,2,4,4,4.0,3.666667,1.666667,19.333333,7.333333,2.804762
100,2022-09-03,17:30,Matchweek 6,Sat,Away,D,1.0,1.0,Aston Villa,2.1,...,17,5,4,4,4.0,4.333333,1.666667,18.666667,8.000000,2.529630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,2024-02-05,20:00,Matchweek 23,Mon,Away,W,3.0,1.0,Brentford,2.1,...,20,0,4,4,0.0,2.666667,1.000000,19.666667,6.333333,3.484848
1228,2024-02-10,12:30,Matchweek 24,Sat,Home,W,2.0,0.0,Everton,1.6,...,12,5,4,4,1.0,3.000000,1.333333,22.000000,10.000000,2.540404
1242,2024-02-17,17:30,Matchweek 25,Sat,Home,D,1.0,1.0,Chelsea,2.6,...,17,5,4,4,2.0,2.666667,0.666667,19.333333,7.333333,3.833333
1256,2024-02-20,19:30,Matchweek 18,Tue,Home,W,1.0,0.0,Brentford,2.3,...,19,1,4,4,2.0,2.000000,0.666667,25.000000,7.666667,4.733333


In [31]:
# Apply the rolling mean function to each group
matches_rolling = matches.groupby("team").apply(lambda x: rolling_mean(x, cols, new_cols, 10))
matches_rolling

C:\Users\edes1\AppData\Local\Temp\ipykernel_4132\914164135.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_mean(x, cols, new_cols, 10))


date   time         round  day venue  \
team                                                                      
Arsenal                 33   2022-08-13  15:00   Matchweek 2  Sat  Home   
                        50   2022-08-20  17:30   Matchweek 3  Sat  Away   
                        61   2022-08-27  17:30   Matchweek 4  Sat  Home   
                        95   2022-08-31  19:30   Matchweek 5  Wed  Home   
                        118  2022-09-04  16:30   Matchweek 6  Sun  Away   
...                                 ...    ...           ...  ...   ...   
Wolverhampton Wanderers 1206 2024-02-04  14:00  Matchweek 23  Sun  Away   
                        1220 2024-02-10  15:00  Matchweek 24  Sat  Home   
                        1243 2024-02-17  15:00  Matchweek 25  Sat  Away   
                        1272 2024-02-25  13:30  Matchweek 26  Sun  Home   
                        1281 2024-03-02  15:00  Matchweek 27  Sat  Away   

                             result   gf   ga        opponent   xg  ...  hour  \
team                                                                ...         
Arsenal                 33        W  4.0  2.0  Leicester City  2.7  ...    15   
                        50        W  3.0  0.0     Bournemouth  1.3  ...    17   
                        61        W  2.0  1.0          Fulham  2.6  ...    17   
                        95        W  2.0  1.0     Aston Villa  2.4  ...    19   
                        118       L  1.0  3.0  Manchester Utd  1.3  ...    16   
...                             ...  ...  ...             ...  ...  ...   ...   
Wolverhampton Wanderers 1206      W  4.0  2.0         Chelsea  2.0  ...    14   
                        1220      L  0.0  2.0       Brentford  1.0  ...    15   
                        1243      W  2.0  1.0       Tottenham  1.8  ...    15   
                        1272      W  1.0  0.0   Sheffield Utd  1.3  ...    13   
                        1281      L  0.0  3.0   Newcastle Utd  0.7  ...    15   

                              day_code  gf_avg_category_codes  \
team                                                            
Arsenal                 33           5                      4   
                        50           5                      4   
                        61           5                      4   
                        95           2                      4   
                        118          6                      4   
...                                ...                    ...   
Wolverhampton Wanderers 1206         6                      3   
                        1220         5                      3   
                        1243         5                      3   
                        1272         6                      3   
                        1281         5                      3   

                             sh_avg_category_codes  \
team                                                 
Arsenal                 33                       3   
                        50                       3   
                        61                       4   
                        95                       4   
                        118                      4   
...                                            ...   
Wolverhampton Wanderers 1206                     1   
                        1220                     2   
                        1243                     2   
                        1272                     2   
                        1281                     2   

                             sh_sot_ratio_avg_category_codes gf_rolling_mean  \
team                                                                           
Arsenal                 33                               3.0            2.00   
                        50                               2.0            3.00   
                        61                               1.0            3.00   
                        95                              

In [32]:
# Drop extra columns
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_avg_category_codes,sh_avg_category_codes,sh_sot_ratio_avg_category_codes,gf_rolling_mean,ga_rolling_mean,sh_rolling_mean,sot_rolling_mean,sh_sot_ratio_rolling_mean
33,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,...,15,5,4,3,3.0,2.00,0.000000,10.000000,2.00,5.000000
50,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,...,17,5,4,3,2.0,3.00,1.000000,14.500000,4.50,3.857143
61,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.6,...,17,5,4,4,1.0,3.00,0.666667,14.333333,5.00,3.349206
95,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.4,...,19,2,4,4,1.0,2.75,0.750000,16.250000,5.75,3.199405
118,2022-09-04,16:30,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,1.3,...,16,6,4,4,2.0,2.60,0.800000,17.400000,6.20,3.109524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,2024-02-04,14:00,Matchweek 23,Sun,Away,W,4.0,2.0,Chelsea,2.0,...,14,6,3,1,0.0,1.70,1.500000,10.600000,4.40,2.491667
1220,2024-02-10,15:00,Matchweek 24,Sat,Home,L,0.0,2.0,Brentford,1.0,...,15,5,3,2,0.0,1.90,1.400000,11.000000,4.50,2.528333
1243,2024-02-17,15:00,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,1.8,...,15,5,3,2,0.0,1.80,1.400000,12.100000,4.70,2.668333
1272,2024-02-25,13:30,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,1.3,...,13,6,3,2,1.0,1.90,1.500000,12.600000,5.00,2.664762


In [33]:
# Assign new index
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,hour,day_code,gf_avg_category_codes,sh_avg_category_codes,sh_sot_ratio_avg_category_codes,gf_rolling_mean,ga_rolling_mean,sh_rolling_mean,sot_rolling_mean,sh_sot_ratio_rolling_mean
0,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,2.7,...,15,5,4,3,3.0,2.00,0.000000,10.000000,2.00,5.000000
1,2022-08-20,17:30,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,1.3,...,17,5,4,3,2.0,3.00,1.000000,14.500000,4.50,3.857143
2,2022-08-27,17:30,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,2.6,...,17,5,4,4,1.0,3.00,0.666667,14.333333,5.00,3.349206
3,2022-08-31,19:30,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,2.4,...,19,2,4,4,1.0,2.75,0.750000,16.250000,5.75,3.199405
4,2022-09-04,16:30,Matchweek 6,Sun,Away,L,1.0,3.0,Manchester Utd,1.3,...,16,6,4,4,2.0,2.60,0.800000,17.400000,6.20,3.109524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,2024-02-04,14:00,Matchweek 23,Sun,Away,W,4.0,2.0,Chelsea,2.0,...,14,6,3,1,0.0,1.70,1.500000,10.600000,4.40,2.491667
1262,2024-02-10,15:00,Matchweek 24,Sat,Home,L,0.0,2.0,Brentford,1.0,...,15,5,3,2,0.0,1.90,1.400000,11.000000,4.50,2.528333
1263,2024-02-17,15:00,Matchweek 25,Sat,Away,W,2.0,1.0,Tottenham,1.8,...,15,5,3,2,0.0,1.80,1.400000,12.100000,4.70,2.668333
1264,2024-02-25,13:30,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,1.3,...,13,6,3,2,1.0,1.90,1.500000,12.600000,5.00,2.664762


In [29]:
# Output the cleaned data
# Save the cleaned data to a new CSV file
matches_rolling.to_csv("../data/processed/rolling_matches.csv", index=False)